# Ship Detection in the Port of Los Angeles (POLA)

In this notebook the labelled dataset
https://www.kaggle.com/rhammell/ships-in-satellite-imagery
is used to train a model to detect the presence of ships in the Port of Los Angeles.

I have also used 
https://towardsdatascience.com/train-image-recognition-ai-with-5-lines-of-code-8ed0bdd8d9ba
as a guide for the training of the machine learning model

In [1]:
# Let's see if it will import:
with open('POLA_vis_images.txt', 'r') as file:  
    POLA_tifs = [tif.rstrip() for tif in file.readlines()]

In [2]:
import json, sys, random
import numpy as np
from imageai.Prediction.Custom import ModelTraining

In [3]:
model_trainer = ModelTraining()
model_trainer.setModelTypeAsResNet()
model_trainer.setDataDirectory("ships_sat")
model_trainer.trainModel(num_objects=2, num_experiments=20, enhance_data=True, batch_size=32, show_network_summary=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 64) 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
max_poolin

Found 846 images belonging to 2 classes.
JSON Mapping for the model classes saved to  ships_sat\json\model_class.json
Number of experiments (Epochs) :  20
Epoch 1/20
 3/55 [>.............................] - ETA: 1:13:01 - loss: 3.2512 - acc: 0.5312

KeyboardInterrupt: 